## Imports

In [1]:
from game import GAME
import torch
import numpy as np

import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass

from torch.distributions import Normal
from utils import config as cfg


from curriculum.reset import reset_random_one_ball

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Instanciando game e exemplificando

In [2]:
env = GAME(draw=True)  
observations = env.reset()

In [3]:
def rewards_function(information):
    colisoes = information['colisoes']
    bolas_caidas = information['bolas_caidas']
    perdeu = information['perdeu']
    ganhou = information['ganhou']
    joga_novamente = information['joga_novamente']
    bolas_jogador = information.get('bolas_jogador', [])
    bolas_adversario = information.get('bolas_adversario', [])
    winner = information.get('winner', None)
    penalizado = information.get('penalizado', False)

    rewards = 0

    if joga_novamente:
        rewards += 1

    if perdeu:
        rewards -= 1.5

    if ganhou:
        rewards += 1.5

    if penalizado:
        rewards -= 1

    return rewards

In [4]:
env.table.draw()

In [5]:
## jogador faz a jogada


#env.iniciou_jogada = False
#while not env.iniciou_jogada:
#    env.table.draw()
#
#obs, information, terminations, rewards = env.step((env.iniciou_jogada_angulo, 
#                                                    env.inicou_jogada_intensidade), rewards_function=rewards_function)
#
#print('state', obs)
#print('termination', terminations)
#print('rewards', rewards)

In [6]:
# computador faz a jogada
print("Iniciando jogada", env.jogador_atual)



angulo = 10
intensidade = 0.9

env.iniciou_jogada = False
obs, information, terminations, rewards = env.step( (angulo, intensidade),  rewards_function=rewards_function)

print("Recompensa da jogada:", rewards)

Iniciando jogada 0
Recompensa da jogada: 0


## Começando treinamento
                           

### ARGS

In [7]:
from models.model_transformers import Model_args, TransformerValueModel, TransformersAtor

### Modelo política

A ideia é que tenhamos uma rede neural como política, especificamente um transformers. 

Queremos que ele passe por cada bola e gere uma representação, no fim a representação do nosso estado será o embedding dabola branca.

In [8]:
from models.model_transformers import Model_args, TransformerValueModel, TransformersAtor

#### Instanciando e testando modelo

Exemplo de foward:

In [9]:
state, bola_branca = (torch.tensor([
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]
        ]),
                      
 torch.tensor([[0.2000, 0.5000],[0.2000, 0.5000]]))

#model(state,bola_branca)

In [10]:
state, bola_branca = (torch.tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.7312, 0.4688, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]]), torch.tensor([[0.6916, 0.8642]]))

print(state.shape, bola_branca.shape)

torch.Size([1, 15, 4]) torch.Size([1, 2])


### Value model

In [11]:
from datetime import datetime
from dataclasses import dataclass, field


@dataclass
class Args:
    exp_name: str = "PPO"
    """the name of this experiment"""
    seed: int = 1
    """seed of the experiment"""
    torch_deterministic: bool = True
    """if toggled, `torch.backends.cudnn.deterministic=False`"""
    cuda: bool = torch.cuda.is_available()
    """if toggled, cuda will be enabled by default"""
    capture_video: bool = False
    """whether to capture videos of the agent performances (check out `videos` folder)"""
    save_model: bool = True
    """whether to save the final model"""

    # Algorithm specific arguments
    total_timesteps: int = 500000
    """total timesteps of the experiments"""
    learning_rate: float = 2.5e-4
    """the learning rate of the optimizer"""
    num_envs: int = 1
    """the number of parallel game environments"""
    num_steps: int = 128
    """the number of steps to run in each environment per policy rollout"""
    anneal_lr: bool = True
    """Toggle learning rate annealing for policy and value networks"""
    gamma: float = 0.99
    """the discount factor gamma"""
    gae_lambda: float = 0.95
    """the lambda for the general advantage estimation"""
    num_minibatches: int = 4
    """the number of mini-batches"""
    update_epochs: int = 4
    """the K epochs to update the policy"""
    norm_adv: bool = True
    """Toggles advantages normalization"""
    clip_coef: float = 0.2
    """the surrogate clipping coefficient"""
    clip_vloss: bool = True
    """Toggles whether or not to use a clipped loss for the value function, as per the paper."""
    ent_coef: float = 0.01
    """coefficient of the entropy"""
    vf_coef: float = 0.5
    """coefficient of the value function"""
    max_grad_norm: float = 0.5
    """the maximum norm for the gradient clipping"""
    target_kl: float = None
    """the target KL divergence threshold"""

    # to be filled in runtime
    batch_size: int = 0
    """the batch size (computed in runtime)"""
    minibatch_size: int = 0
    """the mini-batch size (computed in runtime)"""
    num_iterations: int = 0
    """the number of iterations (computed in runtime)"""
    
    model_args  = None
    
    #model_args: ModelArgs = field(default_factory=ModelArgs)


#PPO

In [12]:
class Agent(nn.Module):
    def __init__(self, action_dim, model_args):
        super().__init__()
        self.critic = TransformerValueModel(model_args)  # Modelo para estimar o valor
        self.actor_mean = TransformersAtor(model_args)  # Modelo para estimar a média das ações
        self.actor_log_std = nn.Parameter(torch.zeros(action_dim))  # Desvio padrão fixo

    def get_value(self, state, bola_branca):
        """Obtém o valor estimado pelo modelo crítico."""
        return self.critic(state, bola_branca)

    def get_action_and_value(self, state, bola_branca, action=None):
        """
        Obtém a ação, log_prob, entropia e valor.
        
        Args:
            x: Entrada para o modelo.
            bola_branca: Dados da bola branca.
            action: Ação opcional para avaliar log_prob e entropia.
        
        Returns:
            action: Ação amostrada ou fornecida.
            log_prob: Logaritmo da probabilidade da ação.
            entropy: Entropia da distribuição.
            value: Valor estimado pelo crítico.
        """
        # O ator retorna ângulo e intensidade
        
        print('state',state.shape)
        print('state',type(state))
        print('bola_branca',bola_branca.shape)
        print('bola_branca',bola_branca.dtype)
        print('bola_branca',bola_branca)
        
        mean = self.actor_mean(state, bola_branca)  # Predição da média das ações
        
        
        log_std = self.actor_log_std.expand_as(mean)  # Ajusta o desvio padrão ao formato correto
        std = torch.exp(log_std)  # Calcula o desvio padrão

        # Distribuição Normal para ações contínuas
        
        dist = Normal(mean, std)

        if action is None:
            action = dist.sample()  # Amostra uma ação da distribuição
            print('action', action.shape)
            
        action[..., 0] = torch.clamp(action[..., 0], 0, 360)
        action[..., 1] = torch.clamp(action[..., 1], 0, 1)
    
        log_prob = dist.log_prob(action).sum(axis=-1)  # Log probabilidade
        entropy = dist.entropy().sum(axis=-1)  # Entropia

        value = self.critic(state, bola_branca)  # Valor estimado pelo crítico

        return action, log_prob, entropy, value

In [13]:
(200, 1) + (60,4)

(200, 1, 60, 4)

In [14]:
torch.zeros((200, 1) + (15,4)).shape

torch.Size([200, 1, 15, 4])

# Test

In [15]:
args = Args()
args.model_args = Model_args(
                    embed_dim  = 128,
                    num_heads  = 8,
                    ff_dim  = 128 * 2,
                    num_layers  =  2,
                    dropout  = 0.1,
                    )
    

In [16]:
from curriculum.reset import reset_random_one_ball

rewards_function

class ambientes():
    
    def __init__(self,n , agent, rewards_function):
        self.n = n
        self.agent = agent
        
        self.envs = [ GAME(draw=True) for _ in range(n)]
        self.rewards_function = rewards_function
        
 
    def format_observations(self, observations : list[tuple] ):
        states = []
        states_white_ball = []
    
        for obs in observations:
            states += [obs[0].unsqueeze(dim=0)]
            states_white_ball += [obs[1].unsqueeze(dim=0)]
        return torch.concat(states) , torch.concat(states_white_ball)
       
    def reset(self):
        states = []
        for env in self.envs:
            states += [reset_random_one_ball(env=env)]
        
        return self.format_observations(states)
            
    def single_observation_space(self):
        return (15,4)
    
    def single_observation_space_white(self):
        return (1,2)
    
    def single_action_space(self):
        return (2,)
    
    
    def step(self, action):
        angulo =  action[:,0]
        intensidade = action[:,1]
        
        
        get_observations, informations , terminations, rewards = [],[],[],[]
        
        for i, env in enumerate(self.envs):

            env_observations, env_informations , env_terminations, env_rewards = env.step( (angulo[i], intensidade[i]), rewards_function =self.rewards_function )
            
            get_observations += [env_observations]
            informations += [env_informations]
            terminations += [env_terminations]
            rewards += [env_rewards]
            
        return self.format_observations(get_observations), informations , terminations, rewards

    
    #def verify_finish_and_reset(self, terminations):
    #    for done , env in zip(terminations, self.envs):
    #        if done:
     
                
#test
#agent = Agent(action_dim=2,model_args=args.model_args)
#a  =  ambientes(2,agent,rewards_function)
#state, balls = a.reset()
#action, logprob, _, value = agent.get_action_and_value(state, balls)
#print(action)
#observations, informations , terminations, rewards = a.step(action)


In [17]:
args.num_envs = 2

In [ ]:
from tqdm import tqdm
import time


def run(args: Args):
    #metrics = defaultdict(list)
    args.batch_size = int(args.num_envs * args.num_steps)
    args.minibatch_size = int(args.batch_size // args.num_minibatches)
    args.num_iterations = args.total_timesteps // args.batch_size
    args.run_name = f"__{args.exp_name}__{args.seed}__{int(time.time())}"

    #writer = SummaryWriter(f"runs/{args.run_name}")
    #writer.add_text(
    #    "hyperparameters",
    #    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    #)

    # seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

    # env setup
    #envs = gym.vector.SyncVectorEnv(
    #    [make_env(args.env_id, i, args.capture_video, args.run_name) for i in range(args.num_envs)],
    #)
    
    agent = Agent(action_dim=2,model_args=args.model_args).to(device)
    
    envs = ambientes(n=args.num_envs, agent=agent,rewards_function=rewards_function)

    optimizer = optim.Adam(agent.parameters(), lr=args.learning_rate, eps=1e-5)


    # ALGO Logic: Storage setup
    obs = torch.zeros((args.num_steps, args.num_envs) + (15,4)).to(device)
    obs_white_ball = torch.zeros((args.num_steps, args.num_envs) + (2,)).to(device)
    
    actions = torch.zeros((args.num_steps, args.num_envs) + (2,)).to(device)
    logprobs = torch.zeros((args.num_steps, args.num_envs)).to(device)
    rewards = torch.zeros((args.num_steps, args.num_envs)).to(device)
    dones = torch.zeros((args.num_steps, args.num_envs)).to(device)
    values = torch.zeros((args.num_steps, args.num_envs)).to(device)




    # start the game
    global_step = 0
    start_time = time.time()
    
    #next_obs, _ = envs.reset(seed=args.seed)
    
    next_obs, next_obs_white = envs.reset()
    next_done = torch.zeros(args.num_envs).to(device)

    # --------------------------------------------------------------------------------

    pbar = tqdm(range(1, args.num_iterations + 1), desc="Iterations")
    for iteration in pbar:
        # annealing the rate if instructed to do so.
        if args.anneal_lr:
            frac = 1.0 - (iteration - 1.0) / args.num_iterations
            lrnow = frac * args.learning_rate
            optimizer.param_groups[0]["lr"] = lrnow

        for step in range(0, args.num_steps):
            global_step += args.num_envs
            obs[step] = next_obs
            obs_white_ball[step] = next_obs_white
            
            dones[step] = next_done

            # action logic
            with torch.no_grad():
                action, logprob, _, value = agent.get_action_and_value(next_obs, next_obs_white)
                values[step] = value.flatten()
                
            actions[step] = action
            logprobs[step] = logprob

            # execute the game and log data.
            (next_obs,next_obs_white), infos, next_done, reward = envs.step(action)


            rewards[step] = torch.tensor(reward).to(device).view(-1)
            next_obs, next_done , next_obs_white = torch.Tensor(next_obs).to(device), torch.Tensor(next_done).to(device), torch.tensor(next_obs_white).to(device)
            
            break
        
            
            # TODO: implementar reset se necessario
            
            
            
            
            
    
            #if "final_info" in infos:
            #    for info in infos["final_info"]:
            #        if info and "episode" in info:
            #            writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
            #            writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)
            #            metrics["charts/episodic_return"].append(info["episode"]["r"])
            #            metrics["charts/episodic_length"].append(info["episode"]["l"])
            #            pbar.set_postfix_str(f"step={global_step}, return={info['episode']['r']}")
            
            
            
            
        """
            PASS OK
        """  



        with torch.no_grad():
            # bootstrap value if not done
            next_value = agent.get_value(next_obs,next_obs_white).reshape(1, -1)
            advantages = torch.zeros_like(rewards).to(device)
            lastgaelam = 0

            # Calculate returns and advantages using GAE
            for t in reversed(range(args.num_steps)):
                if t == args.num_steps - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - dones[t + 1]
                    nextvalues = values[t + 1]
                delta = rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]
                advantages[t] = lastgaelam = delta + args.gamma * args.gae_lambda * lastgaelam
            returns = advantages + values

        # flatten the batch
        
        b_obs = obs.reshape((-1,) + envs.single_observation_space())
        b_obs_white = obs_white_ball.reshape((-1,) + envs.single_observation_space_white()).squeeze(1)
        
        
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1,) + envs.single_action_space())
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        
        
        # Optimizing the policy and value network
        b_inds = np.arange(args.batch_size)
        clipfracs = []
        for epoch in range(args.update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, args.batch_size, args.minibatch_size):
                end = start + args.minibatch_size
                mb_inds = b_inds[start:end]                

                _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_obs_white[mb_inds], b_actions[mb_inds])
                
                print('cheguei')
                
                logratio = b_logprobs[mb_inds] - newlogprob
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clipfracs += [((ratio - 1.0).abs() > args.clip_coef).float().mean().item()]

                mb_advantages = b_advantages[mb_inds]
                if args.norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                # Policy loss
                pg_loss1 = mb_advantages * ratio
                pg_loss2 = mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
                pg_loss = torch.min(pg_loss1, pg_loss2).mean()

                # Value loss
                newvalue = newvalue.view(-1)
                if args.clip_vloss:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(
                        newvalue - b_values[mb_inds],
                        -args.clip_coef,
                        args.clip_coef,
                    )
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                # Entropy Loss
                entropy_loss = entropy.mean()
                loss = pg_loss + args.ent_coef * entropy_loss + v_loss * args.vf_coef

                optimizer.zero_grad()
                loss.backward()
                
                nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
                optimizer.step()

            if args.target_kl is not None and approx_kl > args.target_kl:
                break

        y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
        var_y = np.var(y_true)
        explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

        # TRY NOT TO MODIFY: record rewards for plotting purposes
        """
        writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
        writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
        writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
        writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
        writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
        writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
        writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
        writer.add_scalar("losses/explained_variance", explained_var, global_step)
        writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)
        metrics["losses/value_loss"].append(v_loss.item())
        metrics["losses/policy_loss"].append(pg_loss.item())
        metrics["losses/entropy"].append(entropy_loss.item())
        metrics["losses/SPS"].append(int(global_step / (time.time() - start_time)))
        """

    # --------------------------------------------------------------------------------

    if args.save_model:
        model_path = f"runs/{args.run_name}/{args.exp_name}_model"
        torch.save(agent.state_dict(), model_path)

    #envs.close()
    """
    
    writer.close()
    
    # final evaluation
    eval_episodes = 3
    envs = gym.vector.SyncVectorEnv([make_env(args.env_id, 0, True, args.run_name)])
    agent.eval()
    obs, _ = envs.reset()
    episodic_returns = []
    while len(episodic_returns) < eval_episodes:
        action, logprob, entropy, value = agent.get_action_and_value(torch.Tensor(obs).to(device))
        action = action.cpu().numpy()
        next_obs, _, _, _, infos = envs.step(action)
        obs = next_obs
        if "final_info" in infos:
            for info in infos["final_info"]:
                if "episode" not in info:
                    continue
                print(f"eval_episode={len(episodic_returns)}, episodic_return={info['episode']['r']}")
                episodic_returns += [info["episode"]["r"]]
    
    return metrics
    """

cfg.use_clock = False
args.num_envs = 2
run(args)

c:\CEIA\RL\8pool-with-reinforcement-learning\utils\Ball.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.posicao = torch.tensor(posicao, device=cfg.device, dtype=torch.float32)
Iterations:   0%|          | 0/1953 [00:00<?, ?it/s]

state torch.Size([2, 15, 4])
state <class 'torch.Tensor'>
bola_branca torch.Size([2, 2])
bola_branca torch.float32
bola_branca tensor([[0.8506, 0.5864],
        [0.8942, 0.1138]])





 ENTROU NA REDE
action torch.Size([2, 2])


C:\Users\fazzi\AppData\Local\Temp\ipykernel_36732\3007374644.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_obs, next_done , next_obs_white = torch.Tensor(next_obs).to(device), torch.Tensor(next_done).to(device), torch.tensor(next_obs_white).to(device)
Iterations:   0%|          | 0/1953 [00:03<?, ?it/s]

state torch.Size([64, 15, 4])
state <class 'torch.Tensor'>
bola_branca torch.Size([64, 2])
bola_branca torch.float32
bola_branca tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.00

ValueError: Expected parameter loc (Tensor of shape (64, 2)) of distribution Normal(loc: torch.Size([64, 2]), scale: torch.Size([64, 2])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
  

In [ ]:
#torch.save(b_obs[mb_inds], 'b_obs.pt')
#torch.save(b_obs_white[mb_inds], 'b_obs_white.pt')
#torch.save(b_actions[mb_inds], 'b_actions.pt')
#
#
#_, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_obs_white[mb_inds], b_actions[mb_inds])
#print('cheguei')
                
                
                
# load

b_obs = torch.load('b_obs.pt')
b_obs_white = torch.load('b_obs_white.pt')
b_actions = torch.load('b_actions.pt')

agent = Agent(action_dim=2,model_args=args.model_args)
_, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs, b_obs_white, b_actions)            

C:\Users\fazzi\AppData\Local\Temp\ipykernel_69568\1093790985.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  b_obs = torch.load('b_obs.pt')
C:\Users\fazzi\AppData\Local

### PPO

In [ ]:
action_dim = 2

In [ ]:
agent = Agent(action_dim, args)

(tensor([[0.9769, 0.2516, 1.0000, 0.0000],
         [0.7535, 0.7476, 1.0000, 0.0000],
         [0.7545, 0.4865, 1.0000, 0.0000],
         [0.9280, 0.2381, 1.0000, 0.0000],
         [0.7937, 0.4062, 1.0000, 0.0000],
         [0.8259, 0.5281, 1.0000, 0.0000],
         [0.8699, 0.6038, 1.0000, 0.0000],
         [0.9671, 0.7087, 1.0000, 0.0000],
         [0.0372, 0.3679, 1.0000, 0.0000],
         [0.7445, 0.2964, 1.0000, 0.0000],
         [0.6428, 0.0969, 1.0000, 0.0000],
         [0.8062, 0.4619, 1.0000, 0.0000],
         [0.7291, 0.4671, 1.0000, 0.0000],
         [0.9655, 0.8755, 1.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 0.0000]]),
 tensor([0.4498, 0.4073]))

In [ ]:
action, log_prob, entropy, value = agent.get_action_and_value(torch.concat((state,state)),torch.concat((bola_branca,bola_branca)))

print('action', action)
print('log_prob', log_prob)
print('entropy', entropy)
print('value', value)

action tensor([[ 32.0625,   0.5783],
        [101.6929,   0.6640]])
log_prob tensor([-3.2769, -1.8512], grad_fn=<SumBackward1>)
entropy tensor([2.8379, 2.8379], grad_fn=<SumBackward1>)
value tensor([[0.0828],
        [0.0119]], grad_fn=<AddmmBackward0>)


Aqui vemos que as ações não estão no range esperado, isso porque estamos passando uma distribuiçao normal 

In [ ]:
class PPO:
    def __init__(self, agent, value_model ,lr=1e-3, gamma=0.99, eps_clip=0.2, entropy_coef=0.01):
        self.policy = agent.policy_model
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.entropy_coef = entropy_coef
        
        self.value_model = value_model
        self.value_optimizer = optim.Adam(self.value_model.parameters(), lr=lr)
        
    def compute_advantage(self, rewards, values, masks, gamma):
        returns = []
        R = 0
        for reward, mask in zip(reversed(rewards), reversed(masks)):
            R = reward + gamma * R * mask
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)
        advantage = returns - values
        return advantage.detach(), returns
    
    def update(self, states, bola_branca, actions, log_probs_old, rewards, dones):
        states_flat = states.view(states.size(0), -1)
        batch_size = states.size(0)

        # Calculando valores e vantagens
        values = self.value_model(states_flat, bola_branca).squeeze(-1)
        advantages, returns = self.compute_advantage(rewards, values, 1 - dones, self.gamma)
        
        # Atualizar política
        for _ in range(4):  # Número de atualizações
            outputs = self.policy(states, bola_branca)
            dist = Normal(outputs, torch.ones_like(outputs))  # Normal para distribuição
            log_probs = dist.log_prob(actions).sum(-1)
            entropy = dist.entropy().sum(-1)
            
            # Clipped Surrogate Objective
            ratio = torch.exp(log_probs - log_probs_old)
            surrogate1 = ratio * advantages
            surrogate2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            policy_loss = -torch.min(surrogate1, surrogate2).mean()
            entropy_loss = -self.entropy_coef * entropy.mean()
            
            self.optimizer.zero_grad()
            (policy_loss + entropy_loss).backward()
            self.optimizer.step()

        # Atualizar função de valor
        for _ in range(4):  # Número de atualizações
            values = self.value_model(torch.cat((states_flat, bola_branca), dim=1)).squeeze(-1)
            value_loss = nn.MSELoss()(values, returns)
            
            self.value_optimizer.zero_grad()
            value_loss.backward()
            self.value_optimizer.step()

In [ ]:
policy = Transformers(args)
value = TransformerValueModel(args)

In [ ]:
ppo = PPO(policy, value)

In [ ]:
#

cfg.table_x_max - cfg.table_x_min



800

In [ ]:
import utils.config as cfg


reset_random_one_ball(env)
env.table.draw()

In [ ]:

env.table.draw()

In [ ]:
state_bolas, state_bola_branca = env.get_observations()
# shuffle the balls
state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])]
state_bolas , state_bola_branca

(tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.5000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]),
 tensor([0.7068, 0.3645]))

In [ ]:
def rewards_function(information):
    colisoes = information['colisoes']
    bolas_caidas = information['bolas_caidas']
    perdeu = information['perdeu']
    ganhou = information['ganhou']
    joga_novamente = information['joga_novamente']
    bolas_jogador = information.get('bolas_jogador', [])
    bolas_adversario = information.get('bolas_adversario', [])
    winner = information.get('winner', None)
    penalizado = information.get('penalizado', False)

    rewards = 0

    if penalizado:
        rewards -= 1

    return rewards

In [ ]:
n_episodes = 1000  # Número de episódios para o treinamento
max_steps = 200  # Passos máximos por episódio
gamma = 0.99  # Fator de desconto
learning_rate = 1e-3

In [ ]:
agent = Agent(action_dim=2, args=args)  # Dois valores de ação (ângulo, força)
ppo = PPO(policy_model=agent, value_model=agent.critic, lr=learning_rate, gamma=gamma)

In [ ]:
replay_buffer = {
    "states": [],
    "actions": [],
    "log_probs": [],
    "rewards": [],
    "dones": [],
    "bola_branca": []
}

(tensor([[0.8250, 0.5625, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]),
 tensor([0.0312, 0.7628]))

In [ ]:
agent.get_action_and_value(state, bola_branca)

(tensor([[135.2115,   2.5012]]),
 tensor([-5.0424], grad_fn=<SumBackward1>),
 tensor([2.8379], grad_fn=<SumBackward1>),
 tensor([[-0.3033]], grad_fn=<AddmmBackward0>))

In [ ]:
state, bola_branca

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.7312, 0.4688, 1.0000, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]]]),
 tensor([[0.6916, 0.8642]]))

In [ ]:
state_bolas, state_bola_branca

(tensor([[0.7625, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.5938, 1.0000, 0.0000],
         [0.8250, 0.6250, 1.0000, 0.0000],
         [0.0137, 0.3837, 1.0000, 0.0000],
         [0.8250, 0.4375, 1.0000, 0.0000],
         [0.7625, 0.4375, 1.0000, 0.0000],
         [0.8250, 0.5000, 1.0000, 0.0000],
         [0.7937, 0.5312, 1.0000, 0.0000],
         [0.7625, 0.5000, 1.0000, 0.0000],
         [0.7312, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 0.0000],
         [0.7312, 0.5312, 1.0000, 0.0000],
         [0.7937, 0.4062, 1.0000, 0.0000]]),
 tensor([0.8709, 0.3837]))

In [ ]:
print(state_bolas.shape, state_bola_branca.shape)

torch.Size([15, 4]) torch.Size([2])


In [ ]:
def train():
    
    for episode in range(n_episodes):
        
        reset_random_one_ball(env=env, raio_buraco=13)
        
        rewards_sum = 0
        for step in range(max_steps):
            # Amostra uma ação do agente
            
            
            state_bolas , state_bola_branca = env.get_observations()
            
            action, log_prob, entropy, value = agent.get_action_and_value(state_bolas, state_bola_branca)
            # Executa a ação no ambiente
            
            print(action)
            
            angulo = action[0, 0].item()
            intensidade = action[0, 1].item()
            
            next_obs, information, done, reward = env.step((angulo, intensidade), rewards_function=rewards_function)

            next_state_bolas, next_state_bola_branca = next_obs
            
            # Armazenar dados no buffer
            replay_buffer["states"].append(state_bolas)
            replay_buffer["actions"].append(action)
            replay_buffer["log_probs"].append(log_prob)
            replay_buffer["rewards"].append(reward)
            replay_buffer["dones"].append(done)
            replay_buffer["bola_branca"].append(state_bola_branca)

            print(rewards_sum, reward)
            rewards_sum += reward

            # Atualizar estados
            state_bolas = next_state_bolas
            state_bola_branca = next_state_bola_branca

            if done:
                break
        
        # Atualizar a política usando os dados do episódio
        ppo.update(
            states=torch.stack(replay_buffer["states"]),
            bola_branca=torch.stack(replay_buffer["bola_branca"]),
            actions=torch.stack(replay_buffer["actions"]),
            log_probs_old=torch.stack(replay_buffer["log_probs"]),
            rewards=torch.tensor(replay_buffer["rewards"]),
            dones=torch.tensor(replay_buffer["dones"])
        )
        
        # Limpar o buffer para o próximo episódio
        for key in replay_buffer.keys():
            replay_buffer[key] = []

        print(f"Episódio {episode + 1}/{n_episodes}, Recompensa Total: {rewards_sum}")

In [ ]:
train()

/Users/artur/Documents/VsCode/8pool-with-reinforcement-learning/utils/Ball.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.posicao = torch.tensor(posicao, device=cfg.device, dtype=torch.float32)


tensor([[2.1050e+02, 1.0058e-01]])
0 -1


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
env.reset()
state_bolas, state_bola_branca = env.get_observations()
env.table.bolas = env.table.bolas[1:2] + [env.table.bola_branca]
numero_bola = env.table.bolas[0].numero
state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])] 

state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])]        
        
env.table.bolas = env.table.bolas[1:2] + [env.table.bola_branca]

In [ ]:
state_bolas, state_bola_branca

(tensor([[0.7625, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.5938, 1.0000, 0.0000],
         [0.8250, 0.6250, 1.0000, 0.0000],
         [0.0137, 0.3837, 1.0000, 0.0000],
         [0.8250, 0.4375, 1.0000, 0.0000],
         [0.7625, 0.4375, 1.0000, 0.0000],
         [0.8250, 0.5000, 1.0000, 0.0000],
         [0.7937, 0.5312, 1.0000, 0.0000],
         [0.7625, 0.5000, 1.0000, 0.0000],
         [0.7312, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 0.0000],
         [0.7312, 0.5312, 1.0000, 0.0000],
         [0.7937, 0.4062, 1.0000, 0.0000]]),
 tensor([0.8709, 0.3837]))

In [ ]:
torch.tensor([[352.0620,   1.3407]])

RuntimeError: values expected sparse tensor layout but got Strided

In [ ]:
print("Recompensas no buffer:", replay_buffer["rewards"])

Recompensas no buffer: [-1]
